# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.10it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.12it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.64it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.35it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.33it/s]



...

 78%|███████▊  | 18/23 [00:04<00:01,  4.75it/s]

 83%|████████▎ | 19/23 [00:05<00:00,  4.72it/s]

 91%|█████████▏| 21/23 [00:05<00:00,  4.94it/s]

 96%|█████████▌| 22/23 [00:05<00:00,  4.63it/s]

100%|██████████| 23/23 [00:06<00:00,  3.75it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Liam and I am 10 years old. I am in the 5th grade. I love playing sports, reading books, and riding my bike. My favorite sport is soccer, and I have been playing it for 4 years. I also really like to read books about adventure and science fiction. Some of my favorite books are the Harry Potter series, the Diary of a Wimpy Kid series, and the Captain Underpants series. My favorite bike is a red and black mountain bike, and I like to ride it in the woods near my house. When I am older, I want to be a professional soccer player and travel
Prompt: The president of the United States is
Generated text:  a person who has been elected to serve as the head of state and government of the United States. The president serves a four-year term and is responsible for implementing the laws passed by Congress and serving as the commander-in-chief of the armed forces. The president is also the symbol of national unity and is responsible for representing the cou

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Viv

iana

.


I

...

 a

 tool

,

 but

 a

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Janine Smith and I am a midwife and lactation consultant. I am excited to introduce you to the world of breastfeeding support. As a healthcare provider, I understand the challenges that women face when trying to nurse their babies. That's why I'm dedicated to providing you with the best possible care and support throughout your breastfeeding journey.

With over 10 years of experience in midwifery and lactation consulting, I have helped countless women overcome breastfeeding difficulties and achieve their breastfeeding goals. My approach is holistic and compassionate, taking into account the physical, emotional, and social aspects of breastfeeding.

Whether you're a first-time mom or a

Prompt: The capital of France is
Generated text:  also known for its incredible selection of museums, art galleries, and historical landmarks. From the Eiffel Tower to the Louvre, Paris has something to offer for all interests and ages. Here are some of the top

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Tina

 and

 I

'm

...

 from

 over

2

,

000

In [6]:
llm.shutdown()